Import Libraries

In [1]:
from os.path import join
from os import listdir
import os

import json
import numpy as np
from numpy.linalg import inv
from skimage.measure import marching_cubes
from scipy.ndimage import zoom
import cv2
import nibabel as nib
import meshio
import pymeshlab as ml
from pymeshlab.pmeshlab import Mesh as PMLMesh

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "notebook_connected"

Define global variables

In [2]:
WORKDIR = os.getcwd()
ANNOTATION_FOLDER = join(WORKDIR, "annotations")
MOUSE_NIFTI_FILE = join(WORKDIR, "assets", "images", "sagittal_mouse.nii")
MESHES_AND_VOXELS_FOLDER = join(WORKDIR, "data")
IMAGE_FILES = join(WORKDIR, "assets", "images", "nii-images")

Define the affine transformation matrix 

In [3]:
AFFINE = np.array(
    [[ 0.16952001,  0.,          0.,         21.58835983],
    [ 0.,          0.16952001,  0.,         21.69745827],
    [ 0.,          0.,          0.168538,   95.23029327],
    [ 0.,          0.,          0.,          1.        ]])

INV_AFFINE = inv(AFFINE)

Perform automatic calculation of mouse image for each organ

Data structure:
Image name:
- Organ name
    - Mean response
    - Calculated SUV

First, we should set the calibration factor

In [4]:
CALIBRATION_FACTOR = 441.23

Perform DFS through folder structure

In [5]:
def list_files(filepath, filetype):
    paths = []
    for root, dirs, files in os.walk(filepath):
        for file in files:
            if file.lower().endswith(filetype.lower()):
                paths.append(join(root, file))
    return paths

Get calibrated organ images

In [6]:
calibrated_dir = join(MESHES_AND_VOXELS_FOLDER, "calibrated")
organs = {}

for organ_file in listdir(calibrated_dir):
    organ_name = organ_file.split(".")[0]
    organs[organ_name] = nib.load(join(calibrated_dir, organ_file)).get_fdata()

In [7]:
spect_files = list_files(IMAGE_FILES, "ac.nii")
calculations = [["File name", "Stomach", "Liver", "Duodenum", "Jejunum", "Ileum", "Cecum", "Colon", "Rectum"]]

import csv

for nii_path in spect_files:
    img = nib.load(nii_path).get_fdata()
    row = [nii_path.split("\\")[-1].split("/")[-1], 0., 0., 0., 0., 0., 0., 0., 0.]
    for organ_name in organs:
        organ = organs[organ_name]
        contained_response = img[np.where(organ > 50)]
        mean = np.mean(contained_response)
        idx = calculations[0].index(organ_name)
        row[idx] = mean
    calculations.append(row)

In [ ]:
with open(join(MESHES_AND_VOXELS_FOLDER, "measurements.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerows(calculations)

End